# Tutorial

## Installing everything

To install Interact, simply type
```julia
Pkg.add("Interact")
```

in the REPL.

The basic behavior is as follows: Interact provides a series of widgets. Each widget has an output that can be directly inspected or used to trigger some callbacks (i.e. run some code as soon as the widget changes value): the abstract supertype that gives this behavior is called `AbstractObservable`. Let's see this in practice.

## Displaying a widget

In [1]:
using Interact
ui = button()
display(ui)

WebIO._IJuliaInit()

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Press me!"))

Note that `display` works in a [Jupyter notebook](https://github.com/JuliaLang/IJulia.jl) or in [Atom/Juno IDE](https://github.com/JunoLab/Juno.jl).
Interact can also be deployed in Jupyter Lab, but that requires installing an extension first:

In [ ]:
cd(Pkg.dir("WebIO", "assets"))
;jupyter labextension install webio
;jupyter labextension enable webio/jupyterlab_entry

To deploy the app as a standalone Electron window, one would use [Blink.jl](https://github.com/JunoLab/Blink.jl):

In [2]:
using Blink
w = Window()
body!(w, ui);

[ Info: Listening on: 127.0.0.1:2556, thread id: 1


The app can also be served in a webpage via [Mux.jl](https://github.com/JuliaWeb/Mux.jl):

In [9]:
import Pkg; Pkg.add("Mux")

   Resolving package versions...
    Updating `~/.julia/environments/v1.9/Project.toml`
  [a975b10e] + Mux v1.0.2
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [10]:
using Mux
WebIO.webio_serve(page("/", req -> ui), rand(8000:9000)) # serve on a random port

[ Info: Listening on: 0.0.0.0:8476, thread id: 1


HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"0.0.0.0", 8476), "0.0.0.0", "8476", nothing, Sockets.TCPServer(RawFD(58) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x000000014a7b0330)

In [12]:
w = Window()
body!(w, ui);

34
Hello!
35
Hello!
36
Hello!


## Adding behavior
The value of our button can be inspected using `getindex`:

In [13]:
ui[]

36

In the case of a button, the observable represents the number of times it has been clicked: click on it and check the value again.
For now however this button doesn't do anything. This can be changed by adding callbacks to it.

To add some behavior to the widget we can use the `on` construct. `on` takes two arguments, a function and an `AbstractObservable`. As soon as the observable is changed, the function is called with the latest value.

In [14]:
on(println, ui)

ObserverFunction `println` operating on Node{DOM}(DOM(:html, :div), Any[Scope(Node{DOM}(DOM(:html, :button), Any["Press me!"], Dict{Symbol, Any}(:attributes=>Dict{Any, Any}("data-bind"=>"click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}"), :className=>"is-medium button is-primary", :style=>Dict{String, Any}())), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("loading"=>(Observable(false), nothing), "clicks"=>(Observable(36), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/Users/emueller/.julia/packages/Interact/PENUy

If you click again on the button you will see it printing the number of times it has been clicked so far.

*Tip*: anonymous function are very useful in this programming paradigm. For example, if you want the button to say "Hello!" when pressed, you should use:

In [15]:
on(n -> println("Hello!"), ui)

ObserverFunction defined at In[15]:1 operating on Node{DOM}(DOM(:html, :div), Any[Scope(Node{DOM}(DOM(:html, :button), Any["Press me!"], Dict{Symbol, Any}(:attributes=>Dict{Any, Any}("data-bind"=>"click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}"), :className=>"is-medium button is-primary", :style=>Dict{String, Any}())), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("loading"=>(Observable(false), nothing), "clicks"=>(Observable(36), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/Users/emueller/.julia/packages/Inte

37
Hello!
37
Hello!
38
Hello!
38
Hello!


*Tip n. 2*: using the `[]` syntax you can also set the value of the widget:

In [16]:
ui[] = 33;

33
Hello!
33
Hello!


┌ Error: An exception occurred while trying to send a WebIO message to a frontend:
│   exception =
│    MethodError: no method matching isopen(::Blink.AtomShell.WebIOBlinkComm)
│    The applicable method may be too new: running in world age 33495, while current world is 33533.
│    
│    Closest candidates are:
│      isopen(::Blink.AtomShell.WebIOBlinkComm) (method too new to be called from this world context.)
│       @ Blink ~/.julia/packages/Blink/tnO3a/src/AtomShell/webio.jl:41
│      isopen(::Union{Base.DevNull, Core.CoreSTDERR, Core.CoreSTDOUT})
│       @ Base coreio.jl:24
│      isopen(::Union{Base.AsyncCondition, Timer})
│       @ Base asyncevent.jl:160
│      ...
│    
│    Stacktrace:
│     [1] send_message(pool::WebIO.ConnectionPool, connection::Blink.AtomShell.WebIOBlinkComm, msg::Dict{String, Any})
│       @ WebIO ~/.julia/packages/WebIO/8Xm9p/src/connection.jl:126
│     [2] (::WebIO.var"#5#6"{WebIO.ConnectionPool, Dict{String, Any}, Blink.AtomShell.WebIOBlinkComm})()
│  

### Observables: the implementation of a widget's output
The updatable container that only has the output of the widget but not the widget itself is a `Observable` and can be accessede using `observe(ui)`, though it should normally not be necessary to do so.
To learn more about `Observables` and `AbstractObservable`, check out their documentation [here](https://juliagizmos.github.io/Observables.jl/latest/).

## What widgets are there?

Once you have grasped this paradigm, you can play with any of the many widgets available:

In [17]:
filepicker() |> display # value is the path of selected file
textbox("Write here") |> display # value is the text typed in by the user
autocomplete(["Mary", "Jane", "Jack"]) |> display # as above, but you can autocomplete words
checkbox(label = "Check me!") |> display # value is a boolean describing whether it's ticked
toggle(label = "I have read and agreed") |> display # same as a checkbox but styled differently
slider(1:100, label = "To what extent?", value = 33) |> display # value is the number selected

(div { className="field interact-widget" }
    (div { attributes=Dict{SubString{String}, SubString{String}}("style" => "display:flex; align-items:center;") }
    (label { className="" }
      (input { attributes=Dict{Union{String, Symbol}, Any}(:type => "file", Symbol("data-bind") => "event: {change: onFileUpload}", :style => "display: none;") className="file " })
      (span { className="file-cta" }
        (span { className="file-icon" }
          (i { className="fas fa-upload" }))
        (span { className="" }
          "Choose a file...")))
    (span { attributes=Dict("data-bind" => " text: filename() == '' ? 'No file chosen' : filename()") className="file-name" })))

(div { className="field interact-widget" }
    (input { attributes=Dict{Any, Any}(:type => "text", Symbol("data-bind") => "value: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}") placeholder="Write here" className="input " style=Dict{Any, Any}() }))

(div { className="field interact-widget" }
    (div
    (input { attributes=Dict{Union{String, Symbol}, Any}(:list => Symbol("##292"), :type => "text", Symbol("data-bind") => "value: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}") placeholder="" className="input " style=Dict{Any, Any}() })
    (datalist { attributes=Dict{String, Any}("id" => Symbol("##292"), "data-bind" => "foreach : options_js") }
      (option { attributes=Dict("data-bind" => "value : key") }))))

(div { className="field interact-widget" }
    (div { className="field interact-widget" }
    (input { attributes=Dict{Any, Any}(:type => "checkbox", Symbol("data-bind") => "checked: value, valueUpdate: 'change', event: {change: function (){this.changes(this.changes()+1)}}") id="##293" className="is-checkradio " style=Dict{Any, Any}() })
    (label { attributes=Dict{SubString{String}, SubString{String}}("className" => "", "for" => "##293") }
      "Check me!")))

(div { className="field interact-widget" }
    (div { className="field interact-widget" }
    (input { attributes=Dict{Any, Any}(:type => "checkbox", Symbol("data-bind") => "checked: value, valueUpdate: 'change', event: {change: function (){this.changes(this.changes()+1)}}") id="##294" className="switch " style=Dict{Any, Any}() })
    (label { attributes=Dict{SubString{String}, SubString{String}}("className" => "", "for" => "##294") }
      "I have read and agreed")))

(div { className="field interact-widget" }
    (div { className="interact-flex-row interact-widget" }
    (div { className="interact-flex-row-left" }
      (label { className="interact " style=Dict{Any, Any}(:padding => "5px 10px 0px 10px") }
        "To what extent?"))
    (div { className="interact-flex-row-center" }
      (input { max=100 min=1 attributes=Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal") step=1 className="slider slider is-fullwidth" style=Dict{Any, Any}() }))
    (div { className="interact-flex-row-right" }
      (p { attributes=Dict("data-bind" => "text: formatted_val") }))))

As well as the option widgets, that allow to choose among options:

In [ ]:
dropdown(["a", "b", "c"]) |> display # value is option selected
togglebuttons(["a", "b", "c"]) |> display # value is option selected
radiobuttons(["a", "b", "c"]) |> display # value is option selected

The option widgets can also take as input a dictionary (ordered dictionary is preferrable, to avoid items getting scrambled), in which case the label displays the key while the output stores the value:

In [ ]:
s = dropdown(OrderedDict("a" => "Value 1", "b" => "Value 2"))
display(s)

In [ ]:
s[]

## Creating custom widgets

Interact allows the creation of custom composite widgets starting from simpler ones.
Let's say for example that we want to create a widget that has three sliders and a color
that is updated to match the RGB value we gave with the sliders.

In [19]:
import Pkg; Pkg.add("Colors")

   Resolving package versions...
   Installed Colors ─ v0.12.11
    Updating `~/.julia/environments/v1.9/Project.toml`
  [5ae59095] + Colors v0.12.11
    Updating `~/.julia/environments/v1.9/Manifest.toml`
  [5ae59095] ↑ Colors v0.12.10 ⇒ v0.12.11
Precompiling project...
  ✓ Colors
  ✓ Widgets
  ✓ WebIO
  ✓ CSSUtil
  ✓ JSExpr
  ✓ ColorSchemes
  ✓ Knockout
  ✓ Blink
  ✓ InteractBase
  ✓ Interact
  ✓ PlotUtils
  ✓ PlotThemes
  ✓ RecipesPipeline
  ✓ Plots
  ✓ Plots → IJuliaExt
  ✓ Plots → UnitfulExt
  16 dependencies successfully precompiled in 52 seconds. 339 already precompiled.
  9 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


In [20]:
import Colors
using Plots

function mycolorpicker()
    r = slider(0:255, label = "red")
    g = slider(0:255, label = "green")
    b = slider(0:255, label = "blue")
    output = Interact.@map Colors.RGB(&r/255, &g/255, &b/255)
    plt = Interact.@map plot(sin, color = &output)
    wdg = Widget(["r" => r, "g" => g, "b" => b], output = output)
    @layout! wdg hbox(plt, vbox(:r, :g, :b)) ## custom layout: by default things are stacked vertically
end

[ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a]


mycolorpicker (generic function with 1 method)

And now you can simply instantiate the widget with

In [23]:
mycolorpicker()

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Observable(Plot{Plots.GRBackend() n=1}), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["red"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 255, :min => 0, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable(127), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/Users/emueller/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000000014b428010, Task (runnable) @0x000000014b428010), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 141733920768)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0\",\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\

Note the `&r` syntax: it means automatically update the widget as soon as the
slider changes value. See `Interact.@map` for more details.
If instead we wanted to only update the plot when a button is pressed we would do:

In [22]:
function mycolorpicker()
    r = slider(0:255, label = "red")
    g = slider(0:255, label = "green")
    b = slider(0:255, label = "blue")
    update = button("Update plot")
    output = Interact.@map (&update; Colors.RGB(r[]/255, g[]/255, b[]/255))
    plt = Interact.@map plot(sin, color = &output)
    wdg = Widget(["r" => r, "g" => g, "b" => b, "update" => update], output = output)
    @layout! wdg hbox(plt, vbox(:r, :g, :b, :update)) ## custom layout: by default things are stacked vertically
end

mycolorpicker (generic function with 1 method)

## A simpler approach for simpler cases

While the approach sketched above works for all sorts of situations, there is a specific macro to simplify it in some specific case. If you just want to update some result (maybe a plot) as a function of some parameters (discrete or continuous) simply write `@manipulate` before the `for` loop. Discrete parameters will be replaced by `togglebuttons` and continuous parameters by `slider`: the result will be updated as soon as you click on a button or move the slider:

In [24]:
width, height = 700, 300
colors = ["black", "gray", "silver", "maroon", "red", "olive", "yellow", "green", "lime", "teal", "aqua", "navy", "blue", "purple", "fuchsia"]
color(i) = colors[i%length(colors)+1]
ui = @manipulate for nsamples in 1:200,
        sample_step in slider(0.01:0.01:1.0, value=0.1, label="sample step"),
        phase in slider(0:0.1:2pi, value=0.0, label="phase"),
        radii in 0.1:0.1:60
    cxs_unscaled = [i*sample_step + phase for i in 1:nsamples]
    cys = sin.(cxs_unscaled) .* height/3 .+ height/2
    cxs = cxs_unscaled .* width/4pi
    dom"svg:svg[width=$width, height=$height]"(
        (dom"svg:circle[cx=$(cxs[i]), cy=$(cys[i]), r=$radii, fill=$(color(i))]"()
            for i in 1:nsamples)...
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["nsamples"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 200, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable(100), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/Users/emueller/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000000014a9b1dc0, Task (runnable) @0x000000014a9b1dc0), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 11052708912, 433612256814)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\

or, if you want a plot with some variables taking discrete values:

In [25]:
using Plots

x = y = 0:0.1:30

freqs = OrderedDict(zip(["pi/4", "π/2", "3π/4", "π"], [π/4, π/2, 3π/4, π]))

mp = @manipulate for freq1 in freqs, freq2 in slider(0.01:0.1:4π; label="freq2")
    y = @. sin(freq1*x) * sin(freq2*x)
    plot(x, y)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["freq1"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable(OrderedDict[OrderedDict{String, Any}("key" => "pi/4", "val" => 1, "id" => "idLZidauSG"), OrderedDict{String, Any}("key" => "π/2", "val" => 2, "id" => "idgX7euIwt"), OrderedDict{String, Any}("key" => "3π/4", "val" => 3, "id" => "idZEaHnyBY"), OrderedDict{String, Any}("key" => "π", "val" => 4, "id" => "idtcmW4GWv")]), nothing), "index" => (Observable{Any}(1), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/Users/emueller/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000000010e1df9e0, Task (runnable) @0x000000010e1df9e0), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 141733920768)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyB

## Widget layout

To create a full blown web-app, you should learn the layout tools that the CSS framework you are using provides. See for example the [columns](https://bulma.io/documentation/columns/) and [layout](https://bulma.io/documentation/layout/) section of the Bulma docs. You can use [WebIO](https://github.com/JuliaGizmos/WebIO.jl) to create from Julia the HTML required to create these layouts.

However, this can be overwhelming at first (especially for users with no prior experience in web design). A simpler solution is [CSSUtil](https://github.com/JuliaGizmos/CSSUtil.jl), a package that provides some tools to create simple layouts.

In [26]:
loadbutton = filepicker()
hellobutton = button("Hello!")
goodbyebutton = button("Good bye!")
ui = vbox( # put things one on top of the other
    loadbutton,
    hbox( # put things one next to the other
        pad(1em, hellobutton), # to allow some white space around the widget
        pad(1em, goodbyebutton),
    )
)
display(ui)

(div { style=Dict("display" => "flex", "flex-direction" => "column") }
  Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:attributes => Dict{Union{String, Symbol}, Any}(:type => "file", Symbol("data-bind") => "event: {change: onFileUpload}", :style => "display: none;"), :className => "file ")), Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :i), Any[], Dict{Symbol, Any}(:className => "fas fa-upload"))], Dict{Symbol, Any}(:className => "file-icon")), Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any["Choose a file..."], Dict{Symbol, Any}(:className => ""))], Dict{Symbol, Any}(:className => "file-cta"))], Dict{Symbol, Any}(:className => "")), Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => " text: filename() == '' ? 'No file chosen' : filename()"), :className => "file-name"))], Dict{Symbol, Any}(:attributes => Dict{SubString{String}, SubString{String}}("style" => "display:flex; align-items:center;"))), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("filename" => (Observable{Any}(""), nothing), "path" => (Observable{Any}(""), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/emueller/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/all.js"), Asset("css", nothing, "/Users/emueller/.julia/packages/InteractBase/LJXv3/src/../assets/style.css"), Asset("css", nothing, "/Users/emueller/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("filename" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"filename\"]()) ? (this.valueFromJulia[\"filename\"]=true, this.model[\"filename\"](val)) : undefined})")], "path" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"path\"]()) ? (this.valueFromJulia[\"path\"]=true, this.model[\"path\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000000010d66f3a0, Task (runnable) @0x000000010d66f3a0), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 141733920768)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"filename\":WebIO.getval({\"name\":\"filename\",\"scope\":\"11021303573261083084\",\"id\":\"71\",\"type\":\"observable\"}),\"path\":WebIO.getval({

## Update widgets as function of other widgets

Sometimes the full structure of the GUI is not known in advance. For example, let's imagine we want to load a DataFrame and create a button per column. Not to make it completely trivial, as soon as a button is pressed, we want to plot a histogram of the corresponding column.

*Important note*: this app needs to run in Blink, as the browser doesn't allow us to get access to the local path of a file.

We start by adding a `filepicker` to choose the file, and only once we have a file we want to update the GUI. this can be done as follows:

In [27]:
loadbutton = filepicker()
columnbuttons = Observable{Any}(dom"div"())

Observable{Any}(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}()))

`columnbuttons` is the `div` object that will contain all the relevant buttons. it is an `Observable` as we want its value to change over time.
To add behavior, we can use `map!`:

In [29]:
import Pkg; Pkg.add("CSV")

   Resolving package versions...
   Installed WorkerUtilities ─ v1.6.1
   Installed PooledArrays ──── v1.4.3
   Installed FilePathsBase ─── v0.9.21
   Installed SentinelArrays ── v1.4.3
   Installed WeakRefStrings ── v1.4.2
   Installed CSV ───────────── v0.10.14
    Updating `~/.julia/environments/v1.9/Project.toml`
  [336ed68f] + CSV v0.10.14
    Updating `~/.julia/environments/v1.9/Manifest.toml`
  [336ed68f] + CSV v0.10.14
  [48062228] + FilePathsBase v0.9.21
  [2dfb63ee] + PooledArrays v1.4.3
  [91c51154] + SentinelArrays v1.4.3
  [ea10d353] + WeakRefStrings v1.4.2
  [76eceee3] + WorkerUtilities v1.6.1
Precompiling project...
  ✓ WorkerUtilities
  ✓ PooledArrays
  ✓ WeakRefStrings
  ✓ FilePathsBase
  ✓ SentinelArrays
  ✓ CSV
  ✓ Plots → UnitfulExt
  7 dependencies successfully precompiled in 53 seconds. 354 already precompiled.


In [31]:
import Pkg; Pkg.add("DataFrames")

   Resolving package versions...
   Installed InvertedIndices ─ v1.3.0
   Installed DataFrames ────── v1.6.1
    Updating `~/.julia/environments/v1.9/Project.toml`
  [a93c6f00] + DataFrames v1.6.1
    Updating `~/.julia/environments/v1.9/Manifest.toml`
  [a93c6f00] + DataFrames v1.6.1
  [41ab1584] + InvertedIndices v1.3.0
Precompiling project...
  ✓ InvertedIndices
  ✓ DataFrames
  ✓ Latexify → DataFramesExt
  ✓ BangBang → BangBangDataFramesExt
  ✓ MicroCollections
  ✓ Transducers
  ✓ Transducers → TransducersReferenceablesExt
  ✓ Transducers → TransducersDataFramesExt
  ✓ Transducers → TransducersBlockArraysExt
  ✓ FLoops
  ✓ Folds
  ✓ NDTensors
  ✓ Symbolics
  ✓ ITensors
  ✓ ITensorInfiniteMPS
  ✓ Plots
  ✓ Plots → IJuliaExt
  ✓ Plots → UnitfulExt
  18 dependencies successfully precompiled in 65 seconds. 348 already precompiled.
  2 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


In [32]:
using CSV, DataFrames
data = Observable{Any}(DataFrame)
map!(CSV.read, data, loadbutton)

LoadError: ArgumentError: provide a valid sink argument, like `using DataFrames; CSV.read(source, DataFrame)`

Now as soon as a file is uploaded, the `Observable` `data` gets updated with the correct value. Now, as soon as `data` is updated, we want to update our buttons.

In [33]:
function makebuttons(df)
    buttons = button.(names(df))
    dom"div"(hbox(buttons))
end

map!(makebuttons, columnbuttons, data)

LoadError: MethodError: no method matching names(::Type{DataFrame})

[0mClosest candidates are:
[0m  names([91m::Union{DataFrames.DataFrameColumns, DataFrames.DataFrameRows}[39m)
[0m[90m   @[39m [33mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/abstractdataframe/[39m[90m[4miteration.jl:333[24m[39m
[0m  names([91m::Union{DataFrames.DataFrameColumns, DataFrames.DataFrameRows}[39m, [91m::Any[39m)
[0m[90m   @[39m [33mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/abstractdataframe/[39m[90m[4miteration.jl:334[24m[39m
[0m  names([91m::DataFrames.GroupKey[39m)
[0m[90m   @[39m [33mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/groupeddataframe/[39m[90m[4mgroupeddataframe.jl:655[24m[39m
[0m  ...


We are almost done, we only need to add a callback to the buttons. The cleanest way is to do it during button initialization, meaning during our `makebuttons` step:

In [ ]:
using Plots
plt = Observable{Any}(plot()) # the container for our plot
function makebuttons(df)
    buttons = button.(string.(names(df)))
    for (btn, name) in zip(buttons, names(df))
        map!(t -> histogram(df[name]), plt, btn)
    end
    dom"div"(hbox(buttons))
end

To put it all together:

In [ ]:
using CSV, DataFrames, Interact, Plots
loadbutton = filepicker()
columnbuttons = Observable{Any}(dom"div"())
data = Observable{Any}(DataFrame)
plt = Observable{Any}(plot())
map!(CSV.read, data, loadbutton)

function makebuttons(df)
    buttons = button.(string.(names(df)))
    for (btn, name) in zip(buttons, names(df))
        map!(t -> histogram(df[name]), plt, btn)
    end
    dom"div"(hbox(buttons))
end

map!(makebuttons, columnbuttons, data)

ui = dom"div"(loadbutton, columnbuttons, plt)

And now to serve it in Blink:

In [ ]:
using Blink
w = Window()
body!(w, ui)

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*